## Question-3

In [1]:
# @title include the libraries
import pandas as pd
import numpy as np
from numpy import genfromtxt
import pdb
from matplotlib import pyplot as plt
import math
from itertools import combinations

In [2]:
# @title Load the training Data
import numpy as np

def ReadX(path):
    print(f'>>> Reading data from: {path} ...')
    with open(path) as f:
        # only one line that includes everything
        file = f.readlines()

    print(f'#instances: {len(file)}') # 7352 for training set, 2947 for test set

    X_all = []
    for instance in file:
        f = filter(None, instance.split(' '))
        instance_filterd = list(f)
        instance_cleaned = [float(attr.strip()) for attr in instance_filterd]
        X_all.append(instance_cleaned)
    X_all = np.array(X_all)
    print('>>> Reading finished! Data are converted to numpy array.')
    print(f'shape of X: {X_all.shape} ==> each instance has {X_all.shape[1]} attributes.')

    return X_all

def ReadY(path):
    print(f'>>> Reading data from: {path} ...')
    with open(path) as f:
        # only one line that includes everything
        file = f.readlines()

        print(f'#instances: {len(file)}')  # 7352 for training set, 2947 for test set

    y_all = [float(label.strip()) for label in file]
    y_all = np.array(y_all)
    print('>>> Reading finished! Data are converted to numpy array.')
    print(f'shape of y: {y_all.shape}')
    return y_all



# You can change the path of the files
X_train = ReadX('/content/activity_X_train.txt')
y_train = ReadY('/content/activity_y_train.txt')

X_test = ReadX('/content/activity_X_test.txt')
y_test = ReadY('/content/activity_y_test.txt')


# num_features = X_train.shape[0]
# num_training_example = training_data.shape[1]
# num_training_labels = training_labels.shape[0]
# print("num of features =",num_features,"num_training_example=",num_training_example)
# print("num_labels=",num_training_labels)

>>> Reading data from: /content/activity_X_train.txt ...
#instances: 7352
>>> Reading finished! Data are converted to numpy array.
shape of X: (7352, 561) ==> each instance has 561 attributes.
>>> Reading data from: /content/activity_y_train.txt ...
#instances: 7352
>>> Reading finished! Data are converted to numpy array.
shape of y: (7352,)
>>> Reading data from: /content/activity_X_test.txt ...
#instances: 2947
>>> Reading finished! Data are converted to numpy array.
shape of X: (2947, 561) ==> each instance has 561 attributes.
>>> Reading data from: /content/activity_y_test.txt ...
#instances: 2947
>>> Reading finished! Data are converted to numpy array.
shape of y: (2947,)


In [5]:
# @title  Implementation of perceptron Algorithm (Binary Classification)
def perceptron_algo(X:np.asarray,y:np.asarray,max_pass:int=500, w:np.asarray=np.zeros((57,1)),b:np.float32=0):
  mistakes = np.zeros((0)) # to record the mistakes in each iteration
  num_features = X.shape[0]
  num_training_example = X.shape[1]
  for j in range(max_pass):
    mistake_t = 0
    for i in range(num_training_example):
      if(y[i]*(X[:,i][...,np.newaxis].T@w+b).squeeze() <= 0) :
        w = w + y[i]*X[:,i][...,np.newaxis]
        b = b + y[i]
        mistake_t+=1
    mistakes = np.hstack((mistakes,np.asarray([mistake_t])))
    #print(f"mistake-at-{j}",mistake_t)
  return w,b,mistakes

In [6]:
# @title Multiclass-Perceptron (One Vs One), This calls the perceptron_algo for a pair of  labels
def perceptron_algo_multiclass(X:np.asarray,y:np.asarray,w,b,max_pass:int=500,num_classes:int=1):
  num_perceptron_models = math.comb(num_classes, 2)   #(n(6,2))
  class_pairs =  list(combinations(np.arange(1,num_classes+1), 2)) # for 6 classes , this will calculate the pairs = [(1,2),(1,3),(1,4),(1,5),(1,6),(2,3),(2,4),(2,5),(2,6),(3,4),(3,5),(3,6),(4,5),(4,6),(5,6)]
  mistakes = np.zeros((num_perceptron_models,max_pass))
  w_final = np.zeros((num_perceptron_models,X.shape[0],1))
  print("num-perceptron",num_perceptron_models)
  b_final = np.zeros((num_perceptron_models,))
  for i in range(num_perceptron_models):
    class_k = class_pairs[i][0]
    class_k_ = class_pairs[i][1]
    print("Training for the Class pair:",class_k," & ",class_k_)
    # The following three lines of code perform data isolation for binary classification. They filter the training data to include only instances where the label belongs to either class_k or class_k_.
    #This effectively creates a new dataset specifically for training a linear separator that can distinguish between these two classes.
    idx_ = np.argwhere((y == class_k ) | (y == class_k_)).squeeze()
    X_i = X[:,idx_]
    Y_i = y[idx_]
    # Preparing the data for Binary Perceptron Algorithm
    y_i = -1*np.ones([X_i.shape[1],])
    idx = np.argwhere(Y_i == class_k)
    y_i[idx] = 1
    # calling binary perceptron algorithm
    w_i,b_i,mistake_i = perceptron_algo(X_i,y_i,max_pass=500, w=w[i,:,:],b=b[i])
    # storing Weights , bias and mistakes
    w_final[i,:,:] = w_i
    b_final[i] = b_i
    mistakes[i] = mistake_i
    print(f"final-mistake for class-{i+1}:=",mistake_i[-1])
  return w_final,b_final,mistakes

In [7]:
# @title Calling Perceptron Algorithm
X = X_train.T
y = y_train
max_pass_itr = 500
num_classes = 6
num_perceptron_models = math.comb(num_classes, 2)
num_features = X.shape[0]
w,b,mistakes = perceptron_algo_multiclass(X,y,w=np.zeros((num_perceptron_models,num_features,1)),b=np.zeros((num_perceptron_models,)) , max_pass=max_pass_itr,num_classes=6)

num-perceptron 15
Training for the Class pair: 1  &  2
final-mistake for class-1:= 0.0
Training for the Class pair: 1  &  3
final-mistake for class-2:= 0.0
Training for the Class pair: 1  &  4
final-mistake for class-3:= 0.0
Training for the Class pair: 1  &  5
final-mistake for class-4:= 0.0
Training for the Class pair: 1  &  6
final-mistake for class-5:= 0.0
Training for the Class pair: 2  &  3
final-mistake for class-6:= 0.0
Training for the Class pair: 2  &  4
final-mistake for class-7:= 0.0
Training for the Class pair: 2  &  5
final-mistake for class-8:= 0.0
Training for the Class pair: 2  &  6
final-mistake for class-9:= 0.0
Training for the Class pair: 3  &  4
final-mistake for class-10:= 0.0
Training for the Class pair: 3  &  5
final-mistake for class-11:= 0.0
Training for the Class pair: 3  &  6
final-mistake for class-12:= 0.0
Training for the Class pair: 4  &  5
final-mistake for class-13:= 44.0
Training for the Class pair: 4  &  6
final-mistake for class-14:= 0.0
Training f

In [8]:
# @title Calculating Metrices
total_training_error = np.sum(mistakes[:,-1])
print(f"total-training-error=",total_training_error)

total-training-error= 44.0


In [10]:
# @title Evaluation on Testing Dataset
test_data = X_test.T
test_true_label = y_test
mistakes_test = 0
w = w.squeeze()
class_pairs =  list(combinations(np.arange(1,num_classes+1), 2))
for i in range(test_data.shape[1]):
  temp = np.zeros((num_perceptron_models))
  for j in range(num_perceptron_models):
    predicted_test_output = w[j,:]@test_data[:,i]+ b[j]
    if (predicted_test_output>0):
      #print(class_pairs[j][0])
      temp[j] = class_pairs[j][0]
    else:
      temp[j] = class_pairs[j][1]
  # count the maximum occurance of a number in an array , this is to find a class-k which is repeated  most in temp
  # Count occurrences of each unique element
  unique, counts = np.unique(temp, return_counts=True)
  # Find the index of the maximum count
  max_count_index = np.argmax(counts)
  # Get the number that occurs the maximum number of times
  predicted_test_label = unique[max_count_index]
  # checking if mistakes has occured
  if test_true_label[i] != predicted_test_label:
    mistakes_test+=1
print("Total mistakes-in-test-data",mistakes_test)

Total mistakes-in-test-data 148
